In [97]:
import utils, json
import pandas as pd
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer
from umap import UMAP
from hdbscan import HDBSCAN
from ckiptagger import data_utils, construct_dictionary, WS

In [272]:
job = pd.read_csv("data/20231119_job_market_job.csv")
job_id = json.load(open("data/20231119_job_market_dsa_job_id.json", "r"))
job = job[job["id"].isin(job_id)]
job_desc = job["job_desc"].astype(str).tolist()
stopwords = open("data/20231119_job_market_stopwords.txt", "r").read().splitlines()
stopwords+=["\n","\r","\t", ","]

[SentenceTransformer - Pretrained Models
](https://www.sbert.net/docs/pretrained_models.html#pretrained-models)


In [273]:
ws = WS("./data")

In [274]:
words = ws(job_desc, sentence_segmentation=True, segment_delimiter_set = {"?", "。", "!", ";", ":", " ", "\n", "\r", "\t", "", " "})

In [275]:
words = [[i for i in j if i not in stopwords] for j in words]
job_desc_split = [" ".join(i) for i in words]

In [276]:
pd.DataFrame({"word": [j for i in words for j in i]}).\
    groupby(["word"], as_index = False).\
    size().\
    sort_values("size", ascending = False)["word"].\
    tolist()[:500]

['數據',
 '分析',
 '資料',
 '開發',
 '經驗',
 '系統',
 '學習',
 '資料庫',
 '管理',
 '專案',
 '技術',
 '設計',
 '產品',
 '模型',
 '應用',
 '協助',
 '規劃',
 '客戶',
 '問題',
 '提供',
 '解決',
 '服務',
 '溝通',
 '執行',
 '工具',
 '業務',
 '優化',
 '機器',
 '維護',
 'experience ',
 '負責',
 '商業',
 '•\t',
 '研究',
 '流程',
 '建置',
 '程式',
 '平台',
 '架構',
 'business ',
 '方案',
 '演算法',
 'learning ',
 '建立',
 '行銷',
 'machine ',
 '發展',
 '數位',
 '統計',
 'python',
 '報表',
 '整合',
 '資訊',
 'you ',
 '軟體',
 'ai',
 '提出',
 '撰寫',
 '監控',
 'sql',
 '語言',
 '深度',
 '參與',
 '運用',
 'team ',
 '營運',
 'sql ',
 '報告',
 '策略',
 '建議',
 'are ',
 'work ',
 '測試',
 '作業',
 '能夠',
 '目標',
 '行為',
 '決策',
 '包含',
 '科技',
 '視覺化',
 '效能',
 '工程',
 '市場',
 '知識',
 '金融',
 '提升',
 '專業',
 '產業',
 'ai ',
 '廣告',
 '職務',
 '環境',
 '& ',
 'computer ',
 '評估',
 '主管',
 '操作',
 '工程師',
 '追蹤',
 '智慧',
 '預測',
 '事項',
 '研發',
 '維運',
 '完成',
 '自動化',
 '價值',
 '雲端',
 'development ',
 '持續',
 'new ',
 '驗證',
 '文件',
 '邏輯',
 'analysis ',
 '導入',
 '需要',
 '訓練',
 '創新',
 'bi',
 'from ',
 '主要',
 '配合',
 '獨立',
 '經營',
 'develop ',
 '支援',
 '洞察',
 '影像',
 '

In [277]:
for i in range(6):
    print(job_desc_split[i])

 協助 導入 rd  party  solutionf 調校 isp  pipeline 影像 品質 校正 相機 工廠 流程 測試 協助 camera  bsp  ap 共同 開發 相機 流程 功能
國際 軟體 大廠 高科技 製造業 十多 專家 創立 非常 營運 系統 物聯網 應用 設備 信息 自動化 工廠 管理 技術 運用 過去 累積 工廠 管理 及物 聯網 應用 經驗 搭配 自主 研發 ai 機器 學習 技術 平台 協助 合理 成本 快速 地 導入 高 價值 資訊 系統 解決 工廠 管理 問題 提升 工廠 效能 達到 智能化 工廠 目標 招募 數 名 年輕 新血 嚴格 要求 經驗 只要 技術 底子 好 頭腦 靈活 充滿 學習 事物 幹勁 願意 接受 挑戰 抗壓性 高 對於 工業物 聯網 ai 機器 學習 軟體 生命 週期 平台 技術 工業 應用 非常 興趣 都 雖 新創 但 客戶 都 兩岸 大型 製造業 除了 研究 技術 應用 之外 還 習得 一般人 很 難 機會 深入 碰觸 製造業 知識 提供 優渥 起薪 光明 未來 誠摯 邀請 成為 份子 職務 主要 工作會 參與 ai 機器 學習 web-based 平台 開發 基本 要求 html5/css3/es6+ javascript 以下 過 下列 任一 框架 vue/react/angular  vue http 協議 基礎 知識
資料 倉儲 系統 資料 分析 整理 資料 系統 分析 設計 dw  dm  規劃 設計 專案 系統 訪談 分析 開發 測試 作業 規劃 執行 管理
please  apply  via  www qualcomm com/careers  refer  req number  3018388 ====================================================
 seek  passionate  engineer  in-depth  knowledge  algorithm  design  new  image  sensor  technologies  join  camera  systems  camera  systems  team  creates  embedded  imaging  solutions  snapdrago

In [278]:
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")
# umap_model = UMAP(n_neighbors=15, n_components=5, min_dist=0.0, metric="cosine")
# hdbscan_model = HDBSCAN(min_cluster_size=15, metric="euclidean", cluster_selection_method="eom", prediction_data=True)

In [279]:
topic_model = BERTopic(language = "multilingual")

In [280]:
topics, probs = topic_model.fit_transform(sep_job_desc)

In [281]:
topic_model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,755,-1_數據_分析_資料_開發,"[數據, 分析, 資料, 開發, 經驗, 專案, 系統, 模型, 學習, 技術]",[.\t 負責 內部 外部 數據 分析 包括 產業 數據 網路 數據 收集 種 類型 數據 ...
1,0,157,0_資料庫_sql_管理_效能,"[資料庫, sql, 管理, 效能, 監控, 系統, 調校, mysql, 備份, 維護]",[負責 資料庫 管理 權限 控管 維護 資料庫 備份 規劃 執行 資料庫 安全 機制 規劃 ...
2,1,93,1_學習_機器_模型_learning,"[學習, 機器, 模型, learning, machine, ml, 數據, 開發, 技術...",[主要 任務 機器 學習 深度 學習 統計 分析 模型 實作 開發 包含 數據 資料 清洗 ...
3,2,57,2_分析_行銷_數據_行為,"[分析, 行銷, 數據, 行為, 客戶, 策略, 會員, 活動, 追蹤, 客群]",[會員 輪廓 消費 行為 生命 週期 分析 管理 運用 會員 活動 精準 行銷 活動 數據 ...
4,3,53,3_資料_數據_平台_倉儲,"[資料, 數據, 平台, 倉儲, 開發, 維運, 維護, 管理, 監控, 系統]",[蝙蝠 移動 核心 服務 提供 客戶 專屬 私有化 dmp management plat...
5,4,52,4_演算法_image_camera_vision,"[演算法, image, camera, vision, 算法, 影像, algorithm...",[影像 電腦 視覺 機器 學習 演算法 設計 數值 最佳化 演算法 設計 開發 image ...
6,5,45,5_python_經驗_資料_數據,"[python, 經驗, 資料, 數據, 開發, 語言, 分析, 程式, 工具, sql]",[python sql 工具 建置 符合 客戶 分析 pipeline\n 參與 專案 產...
7,6,44,6_experience_ability_responsibilities_development,"[experience, ability, responsibilities, develo...",[* 矽谷 新創 彈性 氣氛 *\n *responsibilities *\n desig...
8,7,41,7_ai_appier_開發_model,"[ai, appier, 開發, model, 技術, 應用, 智慧, 模型, develo...",[正在 尋找 ai 虚擬人 領域 充滿 熱情 專業 知識 人員 作為 ai 資料 工程師 負...
9,8,39,8_資料_經驗_畢業_分析,"[資料, 經驗, 畢業, 分析, 學習, 學歷, 專案, 未來, 統計, 程式]",[學歷 國內外 大學 含 資訊 電子 電機 資管 工管 數學 統計 學系 畢業 取得 畢業 ...


In [282]:
topic_model.visualize_topics(top_n_topics = 100, title = "Major Topic of DSA Job Desc", height = 500, width = 1000)

In [283]:
topic_model.visualize_barchart(top_n_topics = 12)

In [284]:
topic_model.visualize_hierarchy()

In [285]:
hierarchical_topics = topic_model.hierarchical_topics(job_desc)


100%|██████████| 34/34 [00:00<00:00, 552.36it/s]


In [286]:
topic_model.visualize_hierarchy(hierarchical_topics=hierarchical_topics)


In [288]:
print(topic_model.get_topic_tree(hierarchical_topics))

.
├─and_the_to_with_data
│    ├─and_with_of_sql_database
│    │    ├─■──oracle_資料庫_管理_維護_sql ── Topic: 11
│    │    └─■──資料庫_sql_管理_效能_監控 ── Topic: 0
│    └─and_the_to_data_with
│         ├─and_the_to_data_with
│         │    ├─and_the_to_with_of
│         │    │    ├─■──數位_居住_信義_創新_超乎 ── Topic: 34
│         │    │    └─and_the_to_data_of
│         │    │         ├─and_the_to_data_of
│         │    │         │    ├─google_ads_and_the_facebook
│         │    │         │    │    ├─■──google_行銷_社群_領袖_網路 ── Topic: 33
│         │    │         │    │    └─■──媒體_廣告_投放_數據_產品 ── Topic: 26
│         │    │         │    └─and_the_to_data_of
│         │    │         │         ├─and_the_to_data_of
│         │    │         │         │    ├─■──金融_交易_風險_銀行_系統 ── Topic: 22
│         │    │         │         │    └─and_the_to_data_of
│         │    │         │         │         ├─and_the_to_of_with
│         │    │         │         │         │    ├─■──kkday_行銷_數據_電商_旅遊 ── Topic: 9
│         │    │     